<a href="https://colab.research.google.com/github/arthurst38/deep_learning/blob/main/Utilisation_des_CNNs_avec_Keras_sur_des_donn%C3%A9es_d'expression_faciale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Utilisation des CNNs avec Keras sur des données d'expression faciale

## Vérification de l'utilisation de GPU

Allez dans le menu `Exécution > Modifier le type d'execution` et vérifiez que l'on est bien en Python 3 et que l'accélérateur matériel est configuré sur « GPU ».

In [ ]:
!nvidia-smi

## Téléchargement du dataset d'expressions faciales depuis un repo git

In [ ]:
!git clone https://github.com/muxspace/facial_expressions.git
!ls 
!head facial_expressions/data/legend.csv

## Import de TensorFlow et des autres librairies nécessaires

In [ ]:
import itertools
import pathlib
import typing

import cv2
import IPython
import matplotlib.pyplot as plt
import numpy
import pandas
import seaborn
import sklearn.metrics
import sklearn.model_selection
import sklearn.utils
import tensorflow.keras as keras

## Chargement des données

*Après une rapide étude du dataset à l'aide de commandes usuelles du terminal, implémentez les fonctions de chargement du dataset. Vous pourrez utiliser [`pandas.read_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) pour charger un CSV dans une [`pandas.DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).*

In [ ]:
# Votre code ici
def load_data(csv_path: pathlib.Path, shuffle: bool = True
             ) -> typing.Tuple[numpy.ndarray, numpy.ndarray]:
  return numpy.array([]), numpy.array([])


images, labels = load_data(
    pathlib.Path("facial_expressions") / "data" / "legend.csv")

### Solution

In [ ]:
label_names = ["anger", "contempt", "disgust", "fear", "happiness", "sadness",
               "surprise", "neutral"]
label_to_index = {l: i for i, l in enumerate(label_names)}


image_dir_path = pathlib.Path("facial_expressions") / "images"


def load_image(filename: str) -> numpy.ndarray:
    image = cv2.imread(str(image_dir_path / filename))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return cv2.resize(image, (150, 150))


def load_data(csv_path: pathlib.Path, shuffle: bool = True
             ) -> typing.Tuple[numpy.ndarray, numpy.ndarray]:
  df = pandas.read_csv(csv_path)
  images = numpy.stack(df["image"].map(load_image))
  labels = df["emotion"].map(label_to_index).to_numpy(dtype=numpy.int8)
  if shuffle:
    images, labels = sklearn.utils.shuffle(images, labels)
  return images, labels


# load_data(pathlib.Path("facial_expressions") / "data" / "legend.csv")

In [ ]:
images, labels = load_data(
    pathlib.Path("facial_expressions") / "data" / "legend.csv")

## Exploration du dataset

*Affichez les caractéristiques du dataset suivantes :*

- *La taille des données*
- *Le nombre d'exemple dans chaque classe*
- *Quelques exemples annotés*

In [ ]:
# Votre code ici

### Solution

In [ ]:
print(f"Forme des images : {images.shape}")
print(f"Forme des labels : {labels.shape}")


def plot_label_counts(labels: numpy.ndarray) -> None:
  # Plot des décomptes sur la grande figure
  ax = seaborn.countplot(x=labels)
  ax.set_title("Décomptes des différents labels")
  ax.set_ylabel("Décompte")
  ax.set_xlabel("Label")
  ax.set_xticklabels(label_names, rotation=15)
  plt.show()

  # Récupération des décomptes de chaque label
  _, counts = numpy.unique(labels, return_counts=True)

  # Création d'une pandas.DataFrame pour affichage
  df_counts = pandas.DataFrame({"label": sorted(label_to_index.values()),
                                "décompte": counts},
                              index=label_names)
  # Ajout d'une colonne avec les pourcentages
  df_counts["pourcentage"] = (df_counts["décompte"] * 100
                              / df_counts["décompte"].sum())
  IPython.core.display.display(df_counts)


plot_label_counts(labels)

In [ ]:
# Création de la grille de sous-plots. On donne l'argument figsize pour agrandir
# la taille de la figure qui est petite par défaut
f, ax = plt.subplots(5, 5, figsize=(15, 15))

# On choisit 25 indices au hasard, sans replacement (on ne veut pas afficher la
# même image deux fois)
random_indexes = numpy.random.choice(images.shape[0],
                                     size=(5, 5),
                                     replace=False)

for i in range(5):
  for j in range(5):
    # Récupération de l'image et du label
    img_index = random_indexes[i, j]
    image = images[img_index]
    label = label_names[labels[img_index]]

    # Affichage avec matplotlib et sa fonction imshow, très pratique en vision par
    # ordinateur
    ax[i, j].imshow(image)
    ax[i, j].set_title(f"Exemple {img_index} ({label})")
    ax[i, j].axis('off')

## Séparation train/test

Pour pouvoir évaluer notre modèle, il faut mettre de côté une partie des données. On ne les utilisera pas pendant l'apprentissage.

In [ ]:
# Séparation en train et test (la séparation en train/valid est faite
# automatiquement dans model.fit())
# On fera une séparation 80/20 en respectant les proportions des classes
train_images, test_images, train_labels, test_labels = (
    sklearn.model_selection.train_test_split(images,
                                             labels,
                                             test_size=0.2,
                                             stratify=labels))

## Apprentisage

- *Définissez un modèle*
- *Entraînez votre modèle avec `model.fit(…)`*
- *Affichez vos courbes d'apprentissage*

In [ ]:
# Votre code ici
model = keras.models.Sequential()

### Solution

In [ ]:
def build_model() -> keras.models.Model:
  model = keras.models.Sequential()

  conv2d_params = dict(activation="relu", kernel_size=(3, 3))

  model.add(keras.layers.Input(shape=(150, 150, 3)))
  model.add(keras.layers.Conv2D(200, **conv2d_params))
  model.add(keras.layers.Conv2D(200, **conv2d_params))
  model.add(keras.layers.MaxPool2D(5, 5))
  model.add(keras.layers.Conv2D(50, **conv2d_params))
  model.add(keras.layers.Conv2D(50, **conv2d_params))
  model.add(keras.layers.MaxPool2D(3, 3))
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(100, activation="relu"))
  model.add(keras.layers.Dense(50, activation="relu"))
  model.add(keras.layers.Dropout(rate=0.5))
  model.add(keras.layers.Dense(8, activation="softmax"))

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                loss="sparse_categorical_crossentropy",
                metrics=["accuracy"])
  return model


model = build_model()

# Affichage d'un résumé du modèle
model.summary()

In [ ]:
# Apprentissage du modèle
checkpoint_filepath = "unweighted-checkpoint"
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor="val_accuracy",
    mode="max",
    save_best_only=True)

training = model.fit(train_images,
                     train_labels,
                     epochs=10,
                     validation_split=0.30,
                     callbacks=[model_checkpoint_callback],
                     batch_size=32)

model.load_weights(checkpoint_filepath)

# Plot des métriques d'entraînement
def plot_metrics(history) -> None:
  plt.plot(training.history["accuracy"])
  plt.plot(training.history["val_accuracy"])
  plt.title("Accuracy du modèle")
  plt.ylabel("Accuracy")
  plt.xlabel("Epoch")
  plt.legend(["Entraînement", "Validation"], loc="upper left")
  plt.show()

  plt.plot(training.history["loss"])
  plt.plot(training.history["val_loss"])
  plt.title("Perte du modèle")
  plt.ylabel("Perte")
  plt.xlabel("Epoch")
  plt.legend(["Entraînement", "Validation"], loc="upper right")
  plt.show()


plot_metrics(training.history)

## Évaluation des performances en test

*Utilisez `model.evaluate(…)` pour mesurer les performances de votre modèle sur l'ensemble de test.*

In [ ]:
# Votre code ici

### Solution

In [ ]:
model.evaluate(test_images, test_labels, verbose=1)

## Calcul et affichage de la matrice de confusion

*Utilisez [`sklearn.metrics.confusion_matrix`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) pour calculer la matrice de confusion de votre modèle sur les données de test, puis utilisez [`seaborn.heatmap`](https://seaborn.pydata.org/generated/seaborn.heatmap.html) pour l'afficher.*

In [ ]:
# Votre code ici

### Solution

In [ ]:
def evaluate(model: keras.models.Model,
             images: numpy.ndarray,
             labels: numpy.ndarray
            ) -> None:
  model.evaluate(images, labels, verbose=1)
  predictions = numpy.argmax(model.predict(images), axis=-1)
  confusion_matrix = sklearn.metrics.confusion_matrix(predictions,
                                                      labels,
                                                      normalize="true")
  seaborn.heatmap(confusion_matrix,
                  cmap="rocket_r",
                  xticklabels=label_names,
                  yticklabels=label_names,
                  annot=True,
                  fmt=".2f")
  plt.title("Matrice de confusion")
  plt.show()

  plot_label_counts(labels)


evaluate(model, test_images, test_labels)

## Avec des pondération pour les classes

Il est possible de donner un argument `class_weight` à `model.fit`. Cet argument correspond à un facteur multiplicatif dans la loss à appliquer au score de chaque classe.

*Utilisez [`sklearn.utils.class_weight.compute_class_weight`](https://scikit-learn.org/stable/modules/generated/sklearn.utils.class_weight.compute_class_weight.html) pour calculer les poids à attribuer à chaque classe.*

In [ ]:
class_weights = sklearn.utils.class_weight.compute_class_weight(
    "balanced", numpy.unique(train_labels), train_labels)
print(class_weights)

## Clipping des pondérations

Dans un premier temps, il peut être intéressant de vérifier que les poids de classe améliorent les performances du modèle pour les classes sous-représentées sans utiliser toutefois des pondérations trop fortes.

*Utilisez [`numpy.clip`](https://numpy.org/doc/stable/reference/generated/numpy.clip.html) pour restreindre les valeurs des pondérations à $[0, 5]$.*

In [ ]:
# Votre code ici

### Solution

In [ ]:
numpy.clip(class_weights, None, 5, out=class_weights)
print(class_weights)

## Utilisation des pondérations

*Re-contruisez votre modèle et entraînez le avec les pondérations calculées, puis évaluez-le.*

In [ ]:
# Votre code ici

### Solution

In [ ]:
# Construction du modèle
model = build_model()

# Apprentissage du modèle
training = model.fit(train_images,
                     train_labels,
                     epochs=20,
                     validation_split=0.3,
                     class_weight={c: w for c, w in enumerate(class_weights)},
                     batch_size=32)

plot_metrics(training.history)

In [ ]:
evaluate(model, test_images, test_labels)